In [ ]:
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence_transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=584a95c94a8deefc40bba8786c0dd244f20087b957ae419551467b6d244bdfc5
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch import nn
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

In [ ]:
train = pd.read_csv("PATH/train.csv")
train.head()

,sentence1,sentence2,similarity_score
0,A plane is taking off.,An air plane is taking off.,5.00
1,A man is playing a large flute.,A man is playing a flute.,3.80
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80
3,Three men are playing chess.,Two men are playing chess.,2.60
4,A man is playing the cello.,A man seated is playing the cello.,4.25


In [ ]:
val = pd.read_csv("PATH/stsb_multi_mt-dataset-dev.csv")
val.head()

,sentence1,sentence2,similarity_score
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.00
1,A young child is riding a horse.,A child is riding a horse.,4.75
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.00
3,A woman is playing the guitar.,A man is playing guitar.,2.40
4,A woman is playing the flute.,A man is playing a flute.,2.75


In [ ]:
val['similarity_score'] = val['similarity_score'].astype(float)
train['similarity_score'] = train['similarity_score'].astype(float)

### Fine Tuned Model

In [ ]:
word_embedding_model = models.Transformer('distilbert-base-uncased', max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# create evaluation

evaluator_val = EmbeddingSimilarityEvaluator(list(val.sentence1), 
                                         list(val.sentence2), 
                                         list(val.similarity_score),
                                         name='sts-dev')

In [ ]:
# train 
tr_data   = [InputExample(texts=[s[0], s[1]],  label = [s[2]] ) \
                 for s in list(train[ ['sentence1', 'sentence2', 'similarity_score'] ].drop_duplicates().values)]

In [ ]:
train_dataloader = DataLoader(tr_data, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
#Fine-Tuned the model:
model_path = "MODEL_SAVED_PATH"
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          
          evaluator = evaluator_val,  evaluation_steps=100,
          epochs=5, warmup_steps=500,optimizer_params={'lr': 2e-5},
          show_progress_bar=True, 
          save_best_model = True,
          output_path=model_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]